### Get stock prices data from polygon API

In [1]:
#pip install polygon-api-client

In [56]:
from polygon import RESTClient
import datetime as dt

API_key =
client = RESTClient(API_key)

current_time = dt.datetime.now()

In [62]:
def daterange(end_point,start_point):
    
    for n_days in range(int ((end_point - start_point).days)+1):
        yield start_point + timedelta(n_days)
        
    for dt in daterange(start_point, end_point):
        ex_dividend_date =dt.strftime("%Y-%m-%d")
        return ex_dividend_date 




def get_daily_time_buckets(current_time:dt.datetime,
                           max_lookback_years:int,
                           include_weekends:bool):
    
    
    '''
    Generate a series of days between current time
    and max_lookback_years, with include_weekends option
    '''
    
    from datetime import timedelta, date
    
    start_point = date(2015, 12, 20)
    end_point = date(2016, 1, 11)
    
   

In [59]:
r = get_daily_time_buckets(current_time=dt.datetime.now(),max_lookback_years=2,include_weekends=True)

In [60]:
r

In [31]:
from polygon.rest import models
aggs = client.get_aggs(
    "AAPL",
    1,
    "day",
    "2022-04-04",
    "2022-04-04",
)
print(aggs)

BadResponse: {"status":"ERROR","request_id":"d1b5ff0e171583d0f3602b011f90a403","error":"You've exceeded the maximum requests per minute, please wait or upgrade your subscription to continue. https://polygon.io/pricing"}